In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

크로스 벨리데이션(Cross Validation)

앞선 지도 학습 알고리즘에서는 전체 데이터를 학습 데이터와 테스트 데이터로 나누어 모델을 학습 시켰다.

<table align="left" width="700">
    <tr>
        <td colspan="6" style="border: 1px solid;">
            <div style="text-align: center">Total Data</div>
        </td>
    </tr>
    <tr>
        <td colspan="5" style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Test</div>
        </td>
    </tr>
    <tr>
        <td>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td rowspan="5" style="border: 1px solid;">
            <div style="text-align: center">
                파라미터,<br>
                하이퍼파라미터<br>
                설정
            </div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid;">
            <div style="text-align: center">Validation</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Train</div>
        </td>
    </tr>
    <tr>
        <td>
        </td>
    </tr>
    <tr>
        <td colspan="5">
            <div>모형평가</div>
        </td>
        <td style="border: 1px solid;">
            <div style="text-align: center">Test</div>
        </td>
    </tr>
</table>

와인 데이터를 사용해 하이퍼파라미터 튜닝을 위해 크로스 벨리데이션 기법을 활용 와인 종류를 분류하는 모델을 생성하고 학습시킨다.

In [2]:
# 데이터 불러오기
raw_data = datasets.load_wine() # 사이킷런이 제공하는 와인 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)

# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화 하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화 스케일러에 적용한다.

(178, 13) (178,)
(133, 13) (45, 13) (133,) (45,)


모델 생성 후 데이터 학습

In [3]:
from sklearn.svm import SVC # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
# 크로스 벨리데이션을 사용하기 위해 import 한다.
# StratifiedKFold은 일반적인 KFold 크로스 벨리데이션과는 다르게 라벨링 비율을 유지하면서 데이터를 추출한다.
from sklearn.model_selection import StratifiedKFold
# 그리드 서치를 사용하기 위해 import 한다.
# 그리드 서치는 사용자가 지정한 몇 가지 잠재적인 파라미터들의 후보군들의 조합 중에서 가장 best 조합을 찾아준다.
# 사이킷런은 그리드 서치를 제공함으로써 사용자가 파라미터들의 후보군들을 하나하나 대입하면서 loss를 확인하는 작업을
# 그리드 서치가 대신 해주므로 손쉽게 사용할 수 있다.
# 하이퍼파라미터 후보군의 갯수에 비례해서 시간이 늘어나 최적의 조합을 찾는 시간이 오래걸리는 단점이 있다.
from sklearn.model_selection import GridSearchCV

clf = SVC() # 서포트 벡터 머신 모델을 만든다.
# 그리드 서치에 사용할 하이퍼파라미터 후보군을 설정한다.
param_grid = {
    # 서포트 벡터 머신에서 커널은 정확도가 높은 'rbf' 또는 'linear'로 설정한다.
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 
    'C': [0.1, 1, 10, 100]
}
# n_splits 속성으로 학습 데이터를 나눌 개수를 지정하고 shuffle 속성으로 데이터를 섞어서 StratifiedKFold
# 객체를 만든다.
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# 그리드 서치와 크로스 벨리데이션을 적용하지 않은 서포트 벡터 머신 모델을 학습시킨다.
# clf.fit(x_train, y_train)
# estimator 속성으로 그리드 서치를 적용할 모델, param_grid 속성으로 하이퍼파라미터 후보군, cv 속성으로
# StratifiedGroupKFold 객체, scoring 속성으로 모델 평가 방법을 지정해서 그리드 서치 모델을 만든다.
grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=kfold, scoring='accuracy')
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 그리드 서치 모델을 학습시킨다.
grid.fit(x_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='accuracy')

그리드 서치 결과 확인

In [4]:
# cv_results_ 속성으로 그리드 서치 결과(설정한 파라미터, 모델 평가 등)를 확인할 수 있다.
print(grid.cv_results_)

{'mean_fit_time': array([0.00079942, 0.00100036, 0.00079985, 0.00080013, 0.00080018,
       0.00079932, 0.00120344, 0.00100017, 0.00100012, 0.00060039,
       0.00100064, 0.00040035, 0.        , 0.00060019, 0.00040007,
       0.00019999]), 'std_fit_time': array([3.99709754e-04, 4.86280395e-07, 3.99923481e-04, 4.00066660e-04,
       4.00090285e-04, 3.99667276e-04, 3.98367741e-04, 3.98950589e-07,
       6.81059687e-07, 4.90213006e-04, 9.17214587e-07, 4.90330224e-04,
       0.00000000e+00, 4.90057140e-04, 4.89979265e-04, 3.99971008e-04]), 'mean_score_time': array([0.00040088, 0.00060024, 0.00040045, 0.00020013, 0.        ,
       0.00040097, 0.        , 0.        , 0.        , 0.00040002,
       0.        , 0.00019984, 0.00080013, 0.00039983, 0.00020008,
       0.00080023]), 'std_score_time': array([0.00049097, 0.0004901 , 0.00049045, 0.00040026, 0.        ,
       0.00049109, 0.        , 0.        , 0.        , 0.00048992,
       0.        , 0.00039968, 0.00040007, 0.00048969, 0.00040016

In [5]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000799,3.997098e-04,0.000401,0.000491,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.962963,0.962963,0.925926,1.000000,0.923077,0.954986,0.028341,9
1,0.001000,4.862804e-07,0.000600,0.000490,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.925926,1.000000,0.962963,0.961538,1.000000,0.970085,0.027798,1
2,0.000800,3.999235e-04,0.000400,0.000490,0.1,poly,"{'C': 0.1, 'kernel': 'poly'}",0.666667,0.740741,0.814815,0.615385,0.730769,0.713675,0.068007,16
3,0.000800,4.000667e-04,0.000200,0.000400,0.1,sigmoid,"{'C': 0.1, 'kernel': 'sigmoid'}",0.962963,1.000000,0.962963,1.000000,0.923077,0.969801,0.028638,2
4,0.000800,4.000903e-04,0.000000,0.000000,1,linear,"{'C': 1, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,12
5,0.000799,3.996673e-04,0.000401,0.000491,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,3
6,0.001203,3.983677e-04,0.000000,0.000000,1,poly,"{'C': 1, 'kernel': 'poly'}",0.888889,0.962963,1.000000,0.961538,0.884615,0.939601,0.045322,10
7,0.001000,3.989506e-07,0.000000,0.000000,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.925926,0.962963,0.925926,1.000000,0.961538,0.955271,0.027646,7
8,0.001000,6.810597e-07,0.000000,0.000000,10,linear,"{'C': 10, 'kernel': 'linear'}",0.888889,0.962963,0.925926,1.000000,0.846154,0.924786,0.054014,12
9,0.000600,4.902130e-04,0.000400,0.000490,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.925926,0.962963,0.962963,0.961538,1.000000,0.962678,0.023431,3


In [6]:
np.transpose(pd.DataFrame(grid.cv_results_))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
mean_fit_time,0.000799,0.001,0.0008,0.0008,0.0008,0.000799,0.001203,0.001,0.001,0.0006,0.001001,0.0004,0.0,0.0006,0.0004,0.0002
std_fit_time,0.0004,0.0,0.0004,0.0004,0.0004,0.0004,0.000398,0.0,0.000001,0.00049,0.000001,0.00049,0.0,0.00049,0.00049,0.0004
mean_score_time,0.000401,0.0006,0.0004,0.0002,0.0,0.000401,0.0,0.0,0.0,0.0004,0.0,0.0002,0.0008,0.0004,0.0002,0.0008
std_score_time,0.000491,0.00049,0.00049,0.0004,0.0,0.000491,0.0,0.0,0.0,0.00049,0.0,0.0004,0.0004,0.00049,0.0004,0.0004
param_C,0.1,0.1,0.1,0.1,1,1,1,1,10,10,10,10,100,100,100,100
param_kernel,linear,rbf,poly,sigmoid,linear,rbf,poly,sigmoid,linear,rbf,poly,sigmoid,linear,rbf,poly,sigmoid
params,"{'C': 0.1, 'kernel': 'linear'}","{'C': 0.1, 'kernel': 'rbf'}","{'C': 0.1, 'kernel': 'poly'}","{'C': 0.1, 'kernel': 'sigmoid'}","{'C': 1, 'kernel': 'linear'}","{'C': 1, 'kernel': 'rbf'}","{'C': 1, 'kernel': 'poly'}","{'C': 1, 'kernel': 'sigmoid'}","{'C': 10, 'kernel': 'linear'}","{'C': 10, 'kernel': 'rbf'}","{'C': 10, 'kernel': 'poly'}","{'C': 10, 'kernel': 'sigmoid'}","{'C': 100, 'kernel': 'linear'}","{'C': 100, 'kernel': 'rbf'}","{'C': 100, 'kernel': 'poly'}","{'C': 100, 'kernel': 'sigmoid'}"
split0_test_score,0.962963,0.925926,0.666667,0.962963,0.888889,0.925926,0.888889,0.925926,0.888889,0.925926,0.925926,0.888889,0.888889,0.925926,0.888889,0.814815
split1_test_score,0.962963,1.0,0.740741,1.0,0.962963,0.962963,0.962963,0.962963,0.962963,0.962963,1.0,0.962963,0.962963,0.962963,1.0,0.962963
split2_test_score,0.925926,0.962963,0.814815,0.962963,0.925926,0.962963,1.0,0.925926,0.925926,0.962963,0.925926,0.851852,0.925926,0.962963,0.962963,0.888889


In [7]:
# best 스코어와 best 스코어를 낸 하이퍼파라미터를 확인한다.
print(grid.best_index_) # 그리드 서치가 best 스코어를 낸 하이퍼파라미터의 인덱스를 얻어온다.
print(grid.best_params_) # 그리드 서치가 best 스코어를 낸 하이퍼파라미터를 얻어온다.
print(grid.best_score_) # 그리드 서치가 best 스코어를 얻어온다.
print(grid.best_estimator_) # 그리드 서치가 best 스코어를낸 모델을 얻어온다.

1
{'C': 0.1, 'kernel': 'rbf'}
0.9700854700854702
SVC(C=0.1)


In [8]:
# best 스코어를 낸 하이퍼파라미터를 사용한 모델을 얻어와서 최종 모델로 설정한다.
clf = grid.best_estimator_

그리스 서치 결과에서 beat 스코어를 낸 모델에서 크로스 벨리데이션 스코어 확인

In [9]:
from sklearn.model_selection import cross_validate # 크로스 벨리데이션 스코어를 확인하기 위해 import 한다.
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
# cross_validate() 메소드는 scoring 속성에 여러개의 지표를 넣어서 여러개의 지표를 동시에 확인할 때 사용한다.
# estimator 속성에 그리드 서치 모델, X 속성에 학습 데이터, y 속성에 학습 데이터에 따른 레이블,
# cv 속성에 StratifiedKFold 객체, scoring 속성에 확인할 지표를 지정한다.
cv_score = cross_validate(estimator=clf, X=x_train, y=y_train, cv=kfold, scoring=metrics)
for key in cv_score:
    print(key, cv_score[key])

fit_time [0.0010004  0.0010004  0.00100017 0.00200152 0.00200081]
score_time [0.00200009 0.00200057 0.00200081 0.00299954 0.00200152]
test_accuracy [0.92592593 1.         0.96296296 0.96153846 1.        ]
test_precision_macro [0.925      1.         0.96969697 0.96969697 1.        ]
test_recall_macro [0.925      1.         0.96296296 0.95833333 1.        ]
test_f1_macro [0.925      1.         0.96451914 0.96190476 1.        ]


In [10]:
from sklearn.model_selection import cross_val_score # 크로스 벨리데이션 스코어를 확인하기 위해 import 한다.
# cross_val_score() 메소드는 scoring 속성에 한 개의 지표만 넣어서 한 개의 지표만 확인할 때 사용한다.
# cross_val_score() 메소드의 속성은 cross_validate() 메소드와 같다.
cv_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=kfold, scoring='accuracy')
# cv_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=kfold, scoring='precision_macro')
# cv_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=kfold, scoring='recall_macro')
# cv_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=kfold, scoring='f1_macro')
print(cv_score)

[0.92592593 1.         0.96296296 0.96153846 1.        ]


그리스 서치 결과에서 best 스코어를 낸 모델로 테스트 데이터를 예측한다.

In [11]:
# predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


그리스 서치 결과에서 best 스코어를 낸 모델을 평가한다.

In [12]:
# 혼동 행렬
# confusion_matrix() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 혼동 행렬를 만든다.
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


In [13]:
# 분류 리포트
# classification_report() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 분류 리포트를 만든다.
# target_names 속성으로 분류 리포트에 레이블의 실제값을 출력할 수 있다.
# class_report = classification_report(y_test, predict, target_names=raw_data.target_names) 
class_report = classification_report(y_test, predict, target_names=['A', 'B', 'C']) 
print(class_report)

              precision    recall  f1-score   support

           A       1.00      1.00      1.00        16
           B       1.00      1.00      1.00        21
           C       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

